## Somethin' Fishy - Fishing and Optimal Control
    Xander de la Bruere
    Jakob Gertsch
    Sam Layton
    Matt Mella
    Wilson Stoddard
    

### Abstract

We investigate the use of optimal control to maximize the fish harvested by a large fleet of fishing ships. In particular, we seek to use Pontryagin’s Maximum Principle and bang-bang control theory to find the most efficient path for each ship to take given that we can control the ships’ targets.

### Background

When developing a project idea, we wanted to focus on a broad class of problems involving the control of a group of agents moving through space in order to visit points of interest. To illustrate, controlling a fleet of drones photograph potential threats on a battlefield, flying a group of helicopters to dump water on forest fires, and using drones to do do search-and-rescue are all examples of this type of problem.

This project seeks to apply the classical principles and theory of optimal control to the age-old problem of fishing. Our hope is that the results of our work can be generalized to solve optimal control problems involving the coordination of agents visiting points of interest.

Optimizing fishing harvest is of great interest for several reasons. First, the fishing industry is a significant contributor to GDP throughout the world, as evidenced by its estimated global market value of \$277 billion in 2020 [1]. Since the value that the fishing industry adds to the economy is directly related to the amount of fish caught and sold, optimizing fishing harvests is an important problem to solve for those seeking to boost national economies, not to mention the benefits to the commercial fishermen who actually work in the industry.

Second, more optimal fishing would allow more people greater access to the nutritional benefits of seafood consumption. A 2022 study found that 90\% of Americans do not consume seafood at levels recommended by the US Dietary Guidelines for Americans, and that low-income individuals cite high seafood prices as the primary barrier to higher seafood consumption [2]. Finding a way to harvest more fish using the same amount of labor and capital could increase the supply of various types of seafood, driving down prices and increasing access to the health benefits of seafood for low-income individuals.

Finally, optimized fishing can help relieve problems caused by invasive species that plague lakes and rivers throughout the world. For example, many lakes and rivers in the Southeast of the United States still suffer from invasive silver and bighead carp that entered these waters during extensive flooding in the 1970s [3]. The presence of these invasive aquatic species continues to compromise the water quality and aquatic biodiversity of these waters today. Learning to effectively harvest and dispose of these fish would thus ease or perhaps completely remove the strain on these important ecosystems.

Notwithstanding the impact of optimized fishing on economic well-being, health, and aquatic ecosystems  we were surprised to find that no research has focused on the problem of optimizing group fishing efforts, leaving commercial fishermen to solve this problem on their own. While research exists for optimizing fish harvest in fisheries [4], and while other research has investigated the role of optimal control in maneuvering at sea [5], no extant research attempts addresses the issue of optimizing fishing harvest at sea. This analysis seeks to fill this gap in the literature by applying optimal control theory to maximizing the harvest of a group of fishing boats at sea.

### Problem Description

Our objective is to maximize the total amount of fish harvested by a fleet of ships within a given time interval, which we refer to as the optimal fishing problem.

We operate under the assumption that fish are distributed in various fishing grounds across the sea, with each location featuring a normal distribution of fish populations. Additionally, we assume that fishing ships are more likely to locate these grounds the closer they are to them. Therefore, we simplify our model by assuming that fishing can only occur in specific areas of the sea, which we designate as fishing grounds.

Given this assumption, rather than following the classical approach of controlling ship acceleration directly, we test a novel approach to our optimal control method. We instead control which fishing grounds each ship should target, as if we were orchistrating a sophisticated network of attractive forces between ships and fishing grounds. This method ultimately causes each ship to set sail towards a fishing ground that, while not necessarily closest, optimizes the collective harvest among all ships. 

Specifically, we first control the speed of each boat in the fleet. We require this control variable to be within an admissible control region that has zero as a lower bound and a tunable maximum speed parameter as an upper bound. We then channel the speed of each boat by tracking and controlling a vector of attractions between each ship and each fishing ground. Together, these attractions additively determine the ship’s overall direction, as demonstrated in the figure below.

<div style="text-align: center;">
    <img src="visualizations/boat_trajectory.png" alt="This is a PNG image." width=400>
</div>

Thus, the optimal fishing problem we solve is finding (1) the speed control and (2) attraction network that maximizes the collective amount of fish harvested at each step before the time expires.

In order to use Pontryagin's Maximum Principle (PMP) to determine the optimal speed control and attraction network evolution, we construct a customized state-space model to provide mathematical structure for this problem.

We model the ocean as a two-dimensional plane, with the origin at the bottom left corner. We assume there are $N$ fishing ships and $K$ fishing grounds. To model the dynamics of the fishing environment, we incorporate into our state space the horizontal and vertical position of ship $i$, $x_i \in \mathbb{R}^2$. In addition, the amount of fish remaining in each fishing ground $j$, represented by $F_{j}$, is scaled as a percentage relative to full capacity. Each fishing ground is represented as a point corresponding to its center, $f$. This setup is visualized in the diagram below.

<div style="text-align: center;">
    <img src="visualizations/fishing_environment.png" alt="This is a PNG image." width=700>
</div>

In modeling a ship's relative attraction to one fishing ground over another, we want fishing grounds closest to a given ship to have the strongest relative attraction, while fishing grounds far away from a ship have essentially zero attraction. To accomplish this, we first define a scalar term to represent Gaussian vision from each ship $i$ to each fishing ground $j$ as $g_{ij} = e^{-\alpha \lVert f_j - x_i \rVert^2}$. The parameter $\alpha$ governs how quickly a fishing ground disappears out of sight as a function of its distance from the ship.

We then define the unit vector $d_{ij} = \frac{f_j - x_i}{\lVert f_j - x_i \rVert}$, which points in the direction from ship $i$ to fishing ground $j$. Having done so, we now define the attraction force of fishing ground $j$ to ship $i$ as the vector $A_{ij} = u_{ij}g_{ij}d_{ij}F_j$, where $u_{ij}$ is a control variable used to tune the natural attraction force. Note that the natural attraction force increases with $F_j$, the current amount of fish remaining at ground $j$, and Gaussian vision $g_{ij}$. Finally, we define the net force of attraction on ship $i$, the sum of all fishing ground attraction forces, as $s_i = \sum_{j=1}^{K}A_{ij}$.

Having set up the model, we now define our first-order state equations  $\dot{y} \in \mathbb{R}^{2N+K}$ as a concatenation of $\dot{x_i}$ and $\dot{F_j}$:

\begin{align}
\dot{x_i} &= u_{i,k+1}\frac{s_i}{\lVert s_i \rVert} \in \large \mathbb{R}^2 \notag \\
\dot{F_j} &= -rF_j\sum_{i=1}^{N}e^{-\sigma \lVert f_j - x_i \rVert^2} \in \large \mathbb{R}. \notag
\end{align}
Notice that vector $\dot{y}^{\top}$ contains the change in each boat's $x$- and $y$-position coordinates as well as the amount fish remaining at each fishing ground:
\begin{equation}
\dot{y}^{\top} = \left[\dot{x}_{11}, \dot{x}_{12}, \dot{x}_{21}, \dot{x}_{22}, \ldots, \dot{x}_{N1}, \dot{x}_{N2}, \dot{F}_1, \dot{F}_2, \ldots, \dot{F}_K \right]. \notag
\end{equation}

We assume that all ships start from a harbor $h \in \mathbb{R}^2$ and that all fishing grounds start at full capacity. Thus we require initial conditions $x_i(0) = h$ and $F_j(0) = F_0$, while $x_i(t_f)$ and $F_j(t_f)$ are free.

Given the state evolution we defined, we now seek to find a function for our control that maximizes the fishing harvest. To do this, we define our cost functional:
\begin{equation}
J[u] = \int_{0}^{tf} \left(\beta \sum_{i=1}^{N}\sum_{l=1}^{N} \frac{1}{\varepsilon +\lVert x_l - x_i \rVert^2 } + \gamma \sum_{i=1}^{N} \lVert \dot{x_i} \rVert\right)dt + \delta\sum_{j=1}^{K}F_j(t_f). \notag
\end{equation}

The cost functional consists of three main terms, each scaled by a tunable parameter. The first term, $\sum_{i=1}^{N}\sum_{l=1}^{N} \frac{1}{\varepsilon +\lVert x_l - x_i \rVert^2 }$, is large when ships are close together, thus rewarding boats for spreading out rather than converging to the same fishing ground. The second, $\gamma \sum_{i=1}^{N} \lVert \dot{x_i} \rVert$, penalizes curved paths, which are costly in terms of both time and fuel. The last term, $\delta\sum_{j=1}^{K}F_j(t_f)$, penalizes the amount of fish remaining unharvested in the fishing grounds at the end of the time interval.

We construct co-state equation $p_i \in \mathbb{R}^2$, which corresponds to $x_i$. In addition, we define $p_{F_j}$, which corresponds $F_j$. Both co-state equations are examined more closely in the following section. Looking at endpoint conditions, $y$ is free so we let $p_i(t_f) = 0$. Due to our functional taking the Bolza form, we construct our endpoint condition by taking the derivative with respect to $y$, yielding $p_{F_j}(t_f) = -\delta$. Together, these modeling choices lead to the Hamiltonian:
\begin{equation}
H = \sum_{i=1}^{N}p_i \cdot \dot{x_i} + \sum_{j=1}^{K}p_{F_j}\dot{F}_j - \beta \sum_{i=1}^{N}\sum_{l=1}^{N} \frac{1}{\varepsilon +\lVert x_l - x_i \rVert^2 } - \gamma \sum_{i=1}^{N} \lVert \dot{x_i} \rVert. \notag
\end{equation}

In summary, the table below provides a succinct description of each parameter in our model.

| Parameter | Description | Parameter | Description |
| --- | --- | ---| --- |
| $F_0 \in \mathbb{R}$ | Starting quantity of fish in water | $ r \in \mathbb{R} $ | Rate of fish collection |
| $ N \in \mathbb{R} $ | Number of ships in the model |  $ K \in \mathbb{R} $ | Number of fishing grounds in the model |
| $ V \in \mathbb{R} $ | Maximum speed for ships | $ \alpha \in \mathbb{R} $ | Vision of ships |
| $ \sigma \in \mathbb{R} $ | Inverse size of fishing ground | $ h \in \mathbb{R}^{2} $ | Harbor location |
| $ t_0 \in \mathbb{R} $ | Initial time | $ f_j \in \mathbb{R}^{2} $ | Location of fishing ground $ j $ |
| $ t_f \in \mathbb{R} $ | Final time | $ \beta \in \mathbb{R} $ | Crowding penalty |
| $ \gamma \in \mathbb{R} $ | Distance penalty | $ \delta \in \mathbb{R} $ | Unharvested fish penalty |

The following table indicates the variables of the model.

| Variable | Description |
|---|---|
| $ x_i \in \mathbb{R}^{2} $ | Ship location |
| $ u_{ij} \in \mathbb{R} $ | Control on ship $ i $ for fishing ground $ j $ |
| $ F_j \in \mathbb{R} $ | Quantity of fish at ground $ j $ |

### Solution

Some text

<div style="text-align: center;">
    <img src="visualizations/beta0costate.png" alt="This is a PNG image." width=600>
</div>

### Interpretation

### References

[1] Jannik Lindner. “The most surprising fishing industry statistics and trends in 2024,” 2023.

[2] David Love et al. “Affordability influences nutritional quality of seafood consumption among income and race/ethnicity groups in the united states,” 2022.

[3] U.S. Fish and Wildlife Service. “Invasive carp in southeastern waters,” 2021.

[4] Roland Pulch, Bernd Kugelmann. “Robust optimal control of fishing in a three competing species model,” 2018.

[5] Miele et al. “Optimal control of a ship for collision avoidance maneuvers,” 1999.

We give Dr. Evans, Dr. Whitehead, and other instructors at BYU teaching ACME classes permission to share our project as an example of a good project in future classes they teach.

Xander de la Bruere, Jakob Gertsch, Sam Layton, Matt Mella, and Wilson Stoddard